In [77]:
import numpy as np
import pandas as pd
import pandas.io.sql as sql
import pymysql
import thefuzz
import xlwt
import sqlalchemy
import time
import civitatisAPI as civi
import requests
import json
import re
import os

from requests.auth import HTTPBasicAuth
from thefuzz import fuzz
from thefuzz import process
from sqlalchemy import create_engine
from pymysql import*
from IPython.display import clear_output



In [25]:
## Source cPanel
user= 'beder'
password="DEjUYJjxfRSvab3T"
host="server.digizonelabs.com"
databases_schema = []

con=connect(user="beder",password="DEjUYJjxfRSvab3T",host="server.digizonelabs.com",database="beder_appprod")

posts=sql.read_sql('select * from beder_appprod.post',con)
place=sql.read_sql('select * from beder_appprod.place',con)

c:\Users\beder\.conda\envs\ACTIMPLEMENTATION2022\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\beder\.conda\envs\ACTIMPLEMENTATION2022\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [42]:
barcelona_videos =  pd.read_csv('C:\\Users\\beder\Desktop\Data 2022\Projects\Activities Implementation\\videos_bcn')


In [43]:
barcelona_videos=barcelona_videos.drop_duplicates(subset=['place_id'])
barcelona_videos=barcelona_videos.drop_duplicates(subset=['geo'])
barcelona_videos=barcelona_videos[['place_id','name','latitude','longitude','level']]

# Barcelona Specific

In [44]:
barcelona_videos['hotel']=np.where(barcelona_videos['level']=='lodging',1,0)
barcelona_videos

,place_id,name,latitude,longitude,level,hotel
0,19597,Estació de França,41.3848,2.18540,bus_station,0
1,25820,Grand Hotel Central,41.3850,2.17777,spa,0
3,19569,Correos,41.3816,2.18132,post_office,0
4,2781,"Galeries Maldà, Carrer de la Portaferrissa, Ba...",41.3832,2.17334,shopping_mall,0
5,15080,Arxiu Diocesà de Barcelona,41.3838,2.17565,church,0
...,...,...,...,...,...,...
855,25311,Turó de les Tres Creus,41.4124,2.15114,tourist_attraction,0
856,13470,CosmoCaixa Barcelona,41.4132,2.13112,tourist_attraction,0
892,2549,"Parc Güell, Barcelona, Spain",41.4145,2.15269,tourist_attraction,0
899,19092,Observatori Fabra,41.4184,2.12418,tourist_attraction,0


In [45]:
condition  = (barcelona_videos['level'] == 'restaurant') | (barcelona_videos['level'] == 'food') | (barcelona_videos['level'] == 'cafe') | (barcelona_videos['level'] == 'bar')
barcelona_videos['restaurant']=np.where(condition,1,0)
barcelona_videos

,place_id,name,latitude,longitude,level,hotel,restaurant
0,19597,Estació de França,41.3848,2.18540,bus_station,0,0
1,25820,Grand Hotel Central,41.3850,2.17777,spa,0,0
3,19569,Correos,41.3816,2.18132,post_office,0,0
4,2781,"Galeries Maldà, Carrer de la Portaferrissa, Ba...",41.3832,2.17334,shopping_mall,0,0
5,15080,Arxiu Diocesà de Barcelona,41.3838,2.17565,church,0,0
...,...,...,...,...,...,...,...
855,25311,Turó de les Tres Creus,41.4124,2.15114,tourist_attraction,0,0
856,13470,CosmoCaixa Barcelona,41.4132,2.13112,tourist_attraction,0,0
892,2549,"Parc Güell, Barcelona, Spain",41.4145,2.15269,tourist_attraction,0,0
899,19092,Observatori Fabra,41.4184,2.12418,tourist_attraction,0,0


In [46]:
condition  = (barcelona_videos['restaurant'] != 1) &(barcelona_videos['hotel'] != 1)

barcelona_videos['activity']=np.where(condition,1,0)
barcelona_videos

,place_id,name,latitude,longitude,level,hotel,restaurant,activity
0,19597,Estació de França,41.3848,2.18540,bus_station,0,0,1
1,25820,Grand Hotel Central,41.3850,2.17777,spa,0,0,1
3,19569,Correos,41.3816,2.18132,post_office,0,0,1
4,2781,"Galeries Maldà, Carrer de la Portaferrissa, Ba...",41.3832,2.17334,shopping_mall,0,0,1
5,15080,Arxiu Diocesà de Barcelona,41.3838,2.17565,church,0,0,1
...,...,...,...,...,...,...,...,...
855,25311,Turó de les Tres Creus,41.4124,2.15114,tourist_attraction,0,0,1
856,13470,CosmoCaixa Barcelona,41.4132,2.13112,tourist_attraction,0,0,1
892,2549,"Parc Güell, Barcelona, Spain",41.4145,2.15269,tourist_attraction,0,0,1
899,19092,Observatori Fabra,41.4184,2.12418,tourist_attraction,0,0,1


In [48]:
barcelona_videos['level_bool']=1

In [50]:
usr = 'bederapp'
pss ='w5OWhwNPL1tU'
lnag = 'en'
session = civi.Civitatis(username=usr,password=pss)

In [81]:
barcelona_videos=barcelona_videos.reset_index(drop=True)
barcelona_videos.head(20)

,place_id,name,latitude,longitude,level,hotel,restaurant,activity,level_bool
0,19597,Estació de França,41.3848,2.18540,bus_station,0,0,1,1
1,25820,Grand Hotel Central,41.3850,2.17777,spa,0,0,1,1
2,19569,Correos,41.3816,2.18132,post_office,0,0,1,1
3,2781,"Galeries Maldà, Carrer de la Portaferrissa, Ba...",41.3832,2.17334,shopping_mall,0,0,1,1
4,15080,Arxiu Diocesà de Barcelona,41.3838,2.17565,church,0,0,1,1
5,26412,Chocolat-Box,41.3846,2.17518,store,0,0,1,1
6,25559,Marsalada Barcelona,41.3859,2.17941,painter,0,0,1,1
7,1427,"Universitat de Barcelona, Gran Via de les Cort...",41.3867,2.16387,university,0,0,1,1
8,12027,De Cruz Morales,41.3885,2.15452,food,0,1,0,1
9,3789,"Pg. de Gràcia, 26, 08007 Barcelona, Spain",41.3903,2.16779,art_gallery,0,0,1,1


In [84]:
frames=[]
for idx, latitude in enumerate(barcelona_videos['latitude']):
    longitude =  barcelona_videos['longitude'][idx]
    clear_output(wait=True)
    print(barcelona_videos['name'][idx],idx,len(barcelona_videos['place_id']),flush=True)

    activities= pd.DataFrame(session.closeby_act(latitude,longitude,1))
    
    if activities.shape[0] !=0:
        activities= activities.sort_values(by=['score','reviews'],ascending=False)
        activities= activities.drop_duplicates(subset='score',keep = 'first')
        activities= activities.head(5)
        activities['place_id'] = barcelona_videos['place_id'][idx]
        latitude = []
        longitude = []
        for idx, address in enumerate(activities['address']):
            latitude.append(address['latitude'])
            longitude.append(address['longitude'])


    activities['latitude'] = latitude
    activities['longitude'] = longitude


    frames.append(activities)


MUHBA Turó de la Rovira (Battery Viewpoint) 282 283


In [88]:
activities_bcn = pd.concat(frames)
activities_bcn.shape

(1121, 32)

In [94]:
activities_bcn_final = pd.merge(barcelona_videos,activities_bcn,how = 'left', on = 'place_id')
activities_bcn.to_excel('C:\\Users\\beder\Desktop\Data 2022\Projects\Activities Implementation\\bcn_act.xlsx')

In [93]:
activities_bcn

,id,type,lang,destinationId,title,voucherType,relatedActivities,score,reviews,url,...,MinimumPaxPerBooking,infoVoucher,duration,advance,address,accessibility,rates,place_id,latitude,longitude
3,136840.0,1.0,es,35.0,Entrada al Museo de Historia de Cataluña,0.0,"[{'id': 140625, 'lang': 'en'}, {'id': 140626, ...",10.0,1.0,https://www.civitatis.com/es/barcelona/entrada...,...,NaN,"<p>El día de la actividad, debéis canjear el b...","{'min': 45, 'max': 70, 'duration': 70}","{'days': 0, 'hour': None, 'minutes_before': None}","{'address': '', 'shortAddress': 'Plaça de Pau ...","{'code': 0, 'description': None}","[{'id': 0, 'text': 'Entrada', 'categories': [{...",19597.0,41.380682,2.185689
18,2770.0,1.0,es,35.0,Tour por la Barcelona medieval,0.0,"[{'id': 20074, 'lang': 'pt'}, {'id': 37768, 'l...",9.6,247.0,https://www.civitatis.com/es/barcelona/tour-ba...,...,NaN,"<p>10 minutos antes de la hora indicada, debéi...","{'min': 120, 'max': 0, 'duration': 120}","{'days': 0, 'hour': None, 'minutes_before': 1440}","{'address': '', 'shortAddress': 'Plaza de Bere...","{'code': 100, 'description': 'Sí'}","[{'id': 0, 'text': '(Barcelona medieval)Tour e...",19597.0,41.384430,2.177698
0,144249.0,1.0,es,35.0,Free tour por El Born,0.0,[],9.5,70.0,https://www.civitatis.com/es/barcelona/free-to...,...,NaN,"<p>El día de la actividad, nos encontraremos a...","{'min': 105, 'max': 0, 'duration': 105}","{'days': 0, 'hour': None, 'minutes_before': 720}","{'address': '', 'shortAddress': 'Arco de Triun...","{'code': 0, 'description': None}","[{'id': 0, 'text': 'Tour en español', 'categor...",19597.0,41.390970,2.180259
15,3421.0,1.0,es,35.0,Tour en bicicleta por Barcelona,0.0,"[{'id': 5657, 'lang': 'it'}, {'id': 5663, 'lan...",9.4,77.0,https://www.civitatis.com/es/barcelona/tour-bi...,...,1.0,<p>Nos encontraremos a las <strong>10:00 horas...,"{'min': 210, 'max': 0, 'duration': 210}","{'days': 0, 'hour': None, 'minutes_before': 5760}","{'address': '', 'shortAddress': 'Plaza Real.',...","{'code': 200, 'description': 'No es apto para ...","[{'id': 0, 'text': 'Tour en bicicleta', 'categ...",19597.0,41.380108,2.175349
6,75362.0,1.0,es,35.0,Tour en segway por Barcelona,0.0,"[{'id': 75656, 'lang': 'fr'}, {'id': 76025, 'l...",9.3,11.0,https://www.civitatis.com/es/barcelona/tour-se...,...,NaN,<p>Nos encontraremos a la <strong>hora que hay...,"{'min': 120, 'max': 0, 'duration': 120}","{'days': 0, 'hour': None, 'minutes_before': 120}","{'address': '', 'shortAddress': 'Arco del Triu...","{'code': 200, 'description': 'No es apto para ...","[{'id': 0, 'text': 'Tour en segway', 'categori...",19597.0,41.391056,2.180098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,3348.0,1.0,es,35.0,Visita guiada por la Casa Vicens,0.0,"[{'id': 19900, 'lang': 'pt'}, {'id': 82009, 'l...",7.1,9.0,https://www.civitatis.com/es/barcelona/visita-...,...,1.0,<p>Tendréis que estar 10 minutos antes de la h...,"{'min': 90, 'max': 0, 'duration': 90}","{'days': 1, 'hour': '00:00', 'minutes_before':...","{'address': '', 'shortAddress': 'Calle de les ...","{'code': 100, 'description': 'Sí'}","[{'id': 0, 'text': 'Tour en español', 'categor...",25311.0,41.403432,2.150390
0,132707.0,1.0,es,35.0,Free tour por el Parque Güell,0.0,"[{'id': 134677, 'lang': 'pt'}, {'id': 139210, ...",9.1,479.0,https://www.civitatis.com/es/barcelona/free-to...,...,NaN,"<p>El día de la actividad, nos encontraremos a...","{'min': 60, 'max': 0, 'duration': 60}","{'days': 0, 'hour': None, 'minutes_before': 720}","{'address': '', 'shortAddress': 'Ctra. del Car...","{'code': 0, 'description': None}","[{'id': 0, 'text': 'Tour en español', 'categor...",2549.0,41.415084,2.154769
1,826.0,1.0,es,35.0,Visita guiada por el Parque Güell,0.0,"[{'id': 2581, 'lang': 'pt'}, {'id': 2673, 'lan...",8.3,362.0,https://www.civitatis.com/es/barcelona/visita-...,...,1.0,<p>El punto de encuentro es la <strong>entrada...,"{'min': 75, 'max': 0, 'duration': 75}","{'days': 0, 'hour': None, 'minutes_before': 420}","{'addr